## Deep Reinforcement Learning for Stock Market Trading


---

In [1]:
# Import libraries

import math
import random
import numpy as np
import pandas as pd
import tensorflow as tf
import matplotlib.pyplot as plt
import pandas_datareader as data_reader

from tqdm import tqdm_notebook, tqdm
from collections import deque

In [2]:
# Building the AI Trader network

class AI_Trader():
  
  def __init__(self, state_size, action_space=3, model_name="AITrader"): #Stay, Buy, Sell
    
    self.state_size = state_size
    self.action_space = action_space
    self.memory = deque(maxlen=2000)
    self.inventory = []
    self.model_name = model_name
    
    self.gamma = 0.95
    self.epsilon = 1.0
    self.epsilon_final = 0.01
    self.epsilon_decay = 0.995
    
    self.model = self.model_builder()
    
  def model_builder(self):
    
    model = tf.keras.models.Sequential()
    
    model.add(tf.keras.layers.Dense(units=32, activation='relu', input_dim=self.state_size))
    
    model.add(tf.keras.layers.Dense(units=64, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=128, activation='relu'))
    
    model.add(tf.keras.layers.Dense(units=self.action_space, activation='linear'))
    
    model.compile(loss='mse', optimizer=tf.keras.optimizers.Adam(lr=0.001))
    
    return model
  
  def trade(self, state):
    
    if random.random() <= self.epsilon:
      return random.randrange(self.action_space)
    
    actions = self.model.predict(state)
    return np.argmax(actions[0])
  
  
  def batch_train(self, batch_size):
    
    batch = []
    for i in range(len(self.memory) - batch_size + 1, len(self.memory)):
      batch.append(self.memory[i])
      
    for state, action, reward, next_state, done in batch:
      reward = reward
      if not done:
        reward = reward + self.gamma * np.amax(self.model.predict(next_state)[0])
        
      target = self.model.predict(state)
      target[0][action] = reward
      
      self.model.fit(state, target, epochs=1, verbose=0)
      
    if self.epsilon > self.epsilon_final:
      self.epsilon *= self.epsilon_decay

In [3]:
# Dataset preprocessing

# Defining helper functions

# Sigmoid

def sigmoid(x):
  return 1 / (1 + math.exp(-x))

In [4]:
# Price format function

def stocks_price_format(n):
  if n < 0:
    return "- $ {0:2f}".format(abs(n))
  else:
    return "$ {0:2f}".format(abs(n))

In [5]:
# Dataset loader


def dataset_loader(stock_name):
  
  #Complete the dataset loader function
  dataset = data_reader.DataReader(stock_name, data_source="yahoo")
  
  start_date = str(dataset.index[0]).split()[0]
  end_date = str(dataset.index[-1]).split()[0]
  
  close = dataset['Close']

  return close

In [6]:
# State creator

def state_creator(data, timestep, window_size):
  
  starting_id = timestep - window_size + 1
  
  if starting_id >= 0:
    windowed_data = data[starting_id:timestep+1]
  else:
    windowed_data = - starting_id * [data[0]] + list(data[0:timestep+1])
    
  state = []
  for i in range(window_size - 1):
    state.append(sigmoid(windowed_data[i+1] - windowed_data[i]))
    
  return np.array([state])

In [7]:
# Loading a dataset

stock_name = "HCAT"
data = dataset_loader(stock_name)
data

Date
2019-07-25    39.169998
2019-07-26    38.299999
2019-07-29    41.419998
2019-07-30    43.369999
2019-07-31    44.250000
                ...    
2020-11-13    34.500000
2020-11-16    33.810001
2020-11-17    34.009998
2020-11-18    33.220001
2020-11-19    33.150002
Name: Close, Length: 336, dtype: float64

In [8]:
# Training the AI Trader

# Setting hyper parameters

window_size = 5
episodes = 1

batch_size = 32
data_samples = len(data) - 1

In [9]:
# Defining the Trader model

trader = AI_Trader(window_size)
trader.model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 32)                192       
_________________________________________________________________
dense_1 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_2 (Dense)              (None, 128)               8320      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 387       
Total params: 11,011
Trainable params: 11,011
Non-trainable params: 0
_________________________________________________________________


In [10]:
# Training loop

for episode in range(1, episodes + 1):
  
  print("Episode: {}/{}".format(episode, episodes))
  
  state = state_creator(data, 0, window_size + 1)
  
  total_profit = 0
  trader.inventory = []
  
  for t in tqdm(range(data_samples)):
    
    action = trader.trade(state)
    
    next_state = state_creator(data, t+1, window_size + 1)
    reward = 0
    
    if action == 1: #Buying
      trader.inventory.append(data[t])
      print("AI Trader bought: ", stocks_price_format(data[t]))
      
    elif action == 2 and len(trader.inventory) > 0: #Selling
      buy_price = trader.inventory.pop(0)
      
      reward = max(data[t] - buy_price, 0)
      total_profit += data[t] - buy_price
      print("AI Trader sold: ", stocks_price_format(data[t]), " Profit: " + stocks_price_format(data[t] - buy_price) )
      
    if t == data_samples - 1:
      done = True
    else:
      done = False
      
    trader.memory.append((state, action, reward, next_state, done))
    
    state = next_state
    
    if done:
      print("########################")
      print("TOTAL PROFIT: {}".format(total_profit))
      print("########################")
    
    if len(trader.memory) > batch_size:
      trader.batch_train(batch_size)
      
  if episode % 10 == 0:
    trader.model.save("ai_trader_{}.h5".format(episode))
    

  0%|          | 0/335 [00:00<?, ?it/s]

Episode: 1/1
AI Trader bought:  $ 45.910000
AI Trader sold:  $ 48.630001  Profit: $ 2.720001
AI Trader bought:  $ 48.470001
AI Trader sold:  $ 43.400002  Profit: - $ 5.070000
AI Trader bought:  $ 40.000000
AI Trader sold:  $ 39.860001  Profit: - $ 0.139999
AI Trader bought:  $ 39.220001
AI Trader sold:  $ 36.369999  Profit: - $ 2.850002
AI Trader bought:  $ 34.700001
AI Trader bought:  $ 33.099998


 10%|█         | 34/335 [00:08<05:45,  1.15s/it]

AI Trader sold:  $ 34.040001  Profit: - $ 0.660000


 10%|█         | 35/335 [00:11<08:59,  1.80s/it]

...

 95%|█████████▌| 319/335 [15:23<00:50,  3.17s/it]

AI Trader sold:  $ 35.000000  Profit: $ 3.100000


100%|█████████▉| 334/335 [16:10<00:03,  3.12s/it]

########################
TOTAL PROFIT: -7.044986724853516
########################


100%|██████████| 335/335 [16:13<00:00,  2.91s/it]
